Problem with RAG , is that it is not so efficient when it comes to the retriving work
so if we fine tune our embedding model it will work more acurately

In [1]:
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator

1. Loading the Dataset (visit - https://github.com/ShawhinT/YouTube-Blog/tree/main/LLMs/fine-tuning-embeddings ) to see how the dataset is created.


query | pos | negative
- query is related to job
- pos means if the query matches the job description
- neg means if the query do not matchs the job description
according to the similarities
visit the github to see how this dataset was created. Can take this as an example for making your own dataset for fine tune the embedding model

In [2]:
from datasets import load_dataset

#importing data
dataset = load_dataset("shawhin/ai-job-embedding-finetuning")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/2.27M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/312k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/278k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/809 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/101 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/102 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['query', 'job_description_pos', 'job_description_neg'],
        num_rows: 809
    })
    validation: Dataset({
        features: ['query', 'job_description_pos', 'job_description_neg'],
        num_rows: 101
    })
    test: Dataset({
        features: ['query', 'job_description_pos', 'job_description_neg'],
        num_rows: 102
    })
})

In [4]:
dataset['train'][200]['query']

'Technical Data Analyst criminal justice data accuracy ETL data governance'

In [5]:
dataset['train'][200]['job_description_pos']

'experienced Technical Data Analyst as part of a team that is supporting the law enforcement community in providing data to a state-wide criminal justice data platform. The first of its type in the nation, the platform will integrate data from police departments, the courts, and the state s custodial institutions to better understand and improve pathways through the criminal justice system, reduce recidivism, and improve public reporting of criminal justice data. This role will entail working with the Data Management Team to review CJIS Data and law enforcement data, to include arrest, summons, and citation data for accuracy and facilitating the delivery of work products to the Data Management team.\n\nWho We Are Looking For\n\n6+ years of experience in one or more relational or columnar databases like Oracle, MS SQL Server, PostgreSQL etc. Fluency with SQL data manipulation, preferably using ETL tools and experience with building models in Excel. Able to create dashboards in a BI fram

In [6]:
dataset['train'][200]['job_description_neg']

'requirements and develop solutions that meet those needs.· Design and implement scalable and reliable software architectures that can handle large volumes of data and traffic.· Develop and maintain automated testing frameworks to ensure the quality and reliability of software applications.· Stay up-to-date with the latest AI and cloud-native technologies and trends, and apply them to improve software development processes and outcomes.· Work closely with cross-functional teams, including product managers, designers, and other engineers, to deliver high-quality software products.· Participate in code reviews, design reviews, and other team activities to ensure the quality and consistency of software development practices.· Design and implement cloud-based solutions using Azure services such as Azure Functions, Azure App Service, Azure Storage, and Azure Cosmos DB.· Implement and manage Azure DevOps pipelines for continuous integration and deployment of software applications.· Implement

Importing the model

In [7]:
model_name = "sentence-transformers/all-distilroberta-v1" # acc = 0.88
model = SentenceTransformer(model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

evaluate pre-trained model on eval data

In [8]:
evaluator_valid = TripletEvaluator(
    anchors=dataset["validation"]["query"],
    positives=dataset["validation"]["job_description_pos"],
    negatives=dataset["validation"]["job_description_neg"],
    name="ai-job-validation",
)
evaluator_valid(model)

{'ai-job-validation_cosine_accuracy': 0.8811880946159363}

define loss function

In [9]:
loss = MultipleNegativesRankingLoss(model)

define training args

In [14]:
num_epochs = 3
batch_size = 16
lr = 2e-5
finetuned_model_name = "distilroberta-ai-job-embeddings"

train_args = SentenceTransformerTrainingArguments(
    output_dir=f"models/{finetuned_model_name}",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=lr,
    warmup_ratio=0.1,
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    eval_strategy="steps",
    eval_steps=100,
    logging_steps=100,
)

fine-tune model

In [15]:
%%time
trainer = SentenceTransformerTrainer(
    model=model,
    args=train_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    loss=loss,
    evaluator=evaluator_valid,
)
trainer.train()

Step,Training Loss,Validation Loss,Ai-job-validation Cosine Accuracy
100,0.316900,0.561989,0.990099


CPU times: user 3min 23s, sys: 38.2 s, total: 4min 1s
Wall time: 4min 22s


TrainOutput(global_step=153, training_loss=0.2647241704604205, metrics={'train_runtime': 257.119, 'train_samples_per_second': 9.439, 'train_steps_per_second': 0.595, 'total_flos': 0.0, 'train_loss': 0.2647241704604205, 'epoch': 3.0})

In [16]:
evaluator_test = TripletEvaluator(
    anchors=dataset["test"]["query"],
    positives=dataset["test"]["job_description_pos"],
    negatives=dataset["test"]["job_description_neg"],
    name="ai-job-test",
)
print("Validation:", evaluator_valid(model))
print("Test:", evaluator_test(model))

Validation: {'ai-job-validation_cosine_accuracy': 0.9900990128517151}
Test: {'ai-job-test_cosine_accuracy': 1.0}


Use the Same Model

In [17]:
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import numpy as np

In [18]:
model = SentenceTransformer("shawhin/distilroberta-ai-job-embeddings")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/199 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [19]:
dataset = load_dataset("shawhin/ai-job-embedding-finetuning")

In [20]:
# new query
query = "data scienist 6 year experience, LLMs, credit risk, content marketing"
query_embedding = model.encode(query)

# encode JDs
jd_embeddings = model.encode(dataset["test"]["job_description_pos"])

# compute similarities
similarities = model.similarity(query_embedding, jd_embeddings)
print(similarities.shape)

torch.Size([1, 102])


In [22]:
print(similarities)

tensor([[ 0.0619,  0.0390,  0.1412,  0.0047, -0.0301, -0.0740,  0.2394,  0.0947,
         -0.1053,  0.0136,  0.1197,  0.1683,  0.0212,  0.0556, -0.0879,  0.0056,
          0.0146,  0.1271,  0.1105,  0.1167,  0.1102,  0.1456,  0.1412, -0.0457,
          0.2250,  0.1388,  0.0229,  0.0581, -0.0336,  0.0587,  0.0337,  0.3034,
          0.1930,  0.0994,  0.0408,  0.0155,  0.1188,  0.0702,  0.1504,  0.1529,
          0.1201,  0.0253,  0.1482,  0.0283,  0.0159,  0.1556, -0.0275,  0.0555,
          0.0246,  0.2411,  0.3220,  0.1808,  0.0686, -0.0210, -0.0929,  0.1599,
          0.0126,  0.0415,  0.0432,  0.0006,  0.1517,  0.0581,  0.1165,  0.1860,
         -0.0069,  0.0869,  0.0503,  0.1307, -0.0511,  0.1400,  0.0890,  0.0898,
          0.0786,  0.1055, -0.0725,  0.1166,  0.1072,  0.0256, -0.0619, -0.0337,
          0.0960,  0.1218,  0.0077,  0.1023,  0.0795,  0.1000,  0.0078,  0.1310,
          0.1195, -0.0189,  0.2298,  0.0722,  0.1157, -0.0070,  0.0557,  0.0895,
         -0.0034,  0.1765,  

In [24]:
# sort JDs based on similarity
simiarities_argsorted = np.argsort(similarities.numpy(), axis=1)

# return most similar job description
dataset["test"]["job_description_pos"][int(simiarities_argsorted[0][-1])]

'Skills/Attributes: Data Analysis, Loan Processing, Microsoft Office, SQL Job Description\n**Only those lawfully authorized to work in the designated country associated with the position will be considered.**\n**Please note that all Position start dates and duration are estimates and may be reduced or lengthened based upon a client’s business needs and requirements.**\nRequired Education:• Bachelors/University degree\nRequired Qualifications:• 10+ years of experience in finance/ project management• Experience and proficiency in building data pipelines and performing analytics using KNIME (or similar software• Experience creating team SharePoint sites and maintaining content to make information and documents easily accessible• Proficiency with Visual Basic for Applications (VBA) for Microsoft Office• Proficiency with SQL and relational database management systems• Strong proficiency with Microsoft Excel• Significant experience building end-user tools with Microsoft Access• Additional ex